In [1]:
from pyspark.ml.recommendation import ALS, ALSModel
from pyspark.ml.clustering import KMeansModel
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"
import pyspark.pandas as ps
from pyspark import SparkContext, SparkConf
# means is for items, df is for collab
sc = SparkContext.getOrCreate()
sc.setCheckpointDir('/alscp')
sc.getConf().getAll()

[('spark.driver.host', 'host.docker.internal'),
 ('spark.app.id', 'local-1637648450772'),
 ('spark.executor.memory', '10g'),
 ('spark.executor.id', 'driver'),
 ('spark.app.name', 'pyspark-shell'),
 ('spark.driver.port', '53753'),
 ('spark.driver.memory', '20g'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.submit.pyFiles', ''),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.app.startTime', '1637648450016')]

In [2]:
ps.set_option('compute.default_index_type', 'distributed')
item_path ='data/item_predictions/item_preds.csv' 
collab_path ='data/collab_predictions/collab_preds.csv'

## Verify models can be run

In [3]:
ALS_model = ALSModel.load("models/als_explicit_collab")

In [4]:
kmeans = KMeansModel.load('models/kmeans')

In [5]:
collab_preds = ps.read_csv(collab_path)

In [6]:
collab_preds.head()

,streamerId,userId,interactionTime,prediction
0,641416,86,1.000000,1.623493
1,3050449,94,14.921875,2.215244
2,2469,98,1.000000,NaN
3,3045251,99,5.125000,1.981570
4,3045744,110,1.000000,0.675793


In [7]:
item_preds = ps.read_csv(item_path)

In [8]:
item_preds.head()

,streamerId,avgInteractionTime,interactionCounts,prediction,avgInteractionTimeScaled,interactionCountsScaled
0,2987163,3.338468,71,0,1.290675,0.282402
1,3040638,3.506663,469,7,1.355700,1.865443
2,3050662,5.468750,63,0,2.114256,0.250582
3,3027657,2.075948,323,7,0.802576,1.284730
4,17971,20.078125,6,0,7.762341,0.023865


In [9]:
collab_preds =collab_preds.rename({'prediction': 'collab_preds'}, axis=1)
collab_preds.head()

,streamerId,userId,interactionTime,collab_preds
0,641416,86,1.000000,1.623493
1,3050449,94,14.921875,2.215244
2,2469,98,1.000000,NaN
3,3045251,99,5.125000,1.981570
4,3045744,110,1.000000,0.675793


In [10]:
item_preds =item_preds.rename({'prediction': 'item_preds'}, axis=1)
item_preds.head()

,streamerId,avgInteractionTime,interactionCounts,item_preds,avgInteractionTimeScaled,interactionCountsScaled
0,2987163,3.338468,71,0,1.290675,0.282402
1,3040638,3.506663,469,7,1.355700,1.865443
2,3050662,5.468750,63,0,2.114256,0.250582
3,3027657,2.075948,323,7,0.802576,1.284730
4,17971,20.078125,6,0,7.762341,0.023865


In [11]:
collab_preds.merge(item_preds, on='streamerId', how='inner', suffixes=('_1', '_2'))

,streamerId,userId,interactionTime,collab_preds,avgInteractionTime,interactionCounts,item_preds,avgInteractionTimeScaled,interactionCountsScaled
0,641416,86,1.000000,1.623493,4.683036,7,0,1.810494,0.027842
1,3050449,94,14.921875,2.215244,2.984256,1318,5,1.153734,5.242333
2,2469,98,1.000000,NaN,1.000000,1,0,0.386607,0.003977
3,3045251,99,5.125000,1.981570,3.011341,383,7,1.164205,1.523379
4,3045744,110,1.000000,0.675793,2.424613,194,7,0.937372,0.771633
5,2856504,124,3.635417,3.440084,5.271094,120,7,2.037841,0.477299
6,3051730,175,4.781250,1.774019,2.061919,9684,4,0.797152,38.518025
7,1897415,184,1.000000,0.546613,1.412500,5,0,0.546082,0.019887
8,1227172,184,1.000000,1.294489,1.515625,6,0,0.585951,0.023865
9,3036965,213,1.000000,1.522287,3.895529,411,7,1.506038,1.634749


In [12]:
item_preds.loc[item_preds['streamerId'] == 412]

,streamerId,avgInteractionTime,interactionCounts,item_preds,avgInteractionTimeScaled,interactionCountsScaled
8589967052,412,1.0,1,0,0.386607,0.003977


In [13]:
temp =collab_preds.merge(item_preds, on='streamerId', how='inner', suffixes=('_1', '_2'))
temp.to_spark().coalesce(1).write.format('csv').mode("overwrite").option('header', 'true').save('data/merged_predictions')